# Predicción de la tendecia del ETF SPY

En este proyecto vamos a intentar realizar un modelo que sea capaz de predecir la tendencia diaria del ETF SPY. +

Explciacion que es ETF , SPY, SP500, etc... porque hemos elegido esto, ventajas, desventajas, etc...

Estrucutra e indice del proyecto


In [1]:
import numpy as np  # Numerical operations on arrays and matrices
import pandas as pd  # Data manipulation and analysis
import datetime  # Date and time manipulation
import random  # Generate random numbers and random selection

import pandas_datareader.data as web  # Fetch financial data from online sources

import matplotlib.pyplot as plt  # Data visualization and plotting
import seaborn as sns  # Statistical data visualization

import pandas_ta as ta  # Technical analysis indicators and tools

import statsmodels.api as sm  # Statistical modeling and econometrics

from alpha_vantage.timeseries import TimeSeries  # Fetch time series data from Alpha Vantage API

from statsmodels.stats.outliers_influence import variance_inflation_factor  # Calculate Variance Inflation Factor (VIF) for multicollinearity analysis

from sklearn.model_selection import train_test_split, GridSearchCV  # Data splitting and hyperparameter tuning
from sklearn.preprocessing import MinMaxScaler  # Scaling features to a range
from sklearn.svm import SVC  # Support Vector Classifier
from sklearn.feature_selection import RFE  # Recursive Feature Elimination for feature selection
from sklearn.metrics import accuracy_score, classification_report, RocCurveDisplay, roc_auc_score  # Model evaluation metrics
from sklearn.ensemble import RandomForestClassifier  # Random Forest for classification and feature importance

from scipy import stats  # Statistical functions and tests

from collections import defaultdict  # Dictionary subclass for counting and grouping

from minisom import MiniSom  # Self Organizing Maps for dimensionality reduction and clustering

import warnings
warnings.filterwarnings('ignore') #ignore warnings 

In [ ]:
pip install pandas_datareader pandas_ta statsmodels alpha_vantage minisom

  Using cached pandas_ta-0.3.14b.tar.gz (115 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=b3cb3760509b9e75c13b78751fc0c0df84a7d25d936661d93e7239bf24a8a2a2
  Stored in directory: /Users/ivan/Library/Caches/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
  Created wheel for minisom: filename=MiniSom-2.3.5-py3-none-any.whl size=12031 sha256=dd3253e6cc7c404bcd4b78fc61c7997ccc5a79cdb855e79fe00f4b5286659946
  Stored in directory: /Users/ivan/Library/Caches/pip/wheels/be/33/77/b6bb14008fa79a2dd2b79d565e941d5faa2d7f7526e528345b
Successfully built pandas_ta minisom
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Fetching data with AlphaVantage premium API key
api_key = 'TWABU1U88I5L7AKL'
ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch daily adjusted data for NVDA
#data, meta_data = ts.get_daily_adjusted(symbol='SPY', outputsize='full')
data, meta_data = ts.get_daily(symbol='SPY', outputsize='full')

# Ensure the index is datetime and sort it
data.index = pd.to_datetime(data.index)
data.sort_index(inplace=True)

# Define the date range
start_date = '2018-02-01'
end_date = '2025-02-01'

# Filter data for the specified date range
data = data.loc[start_date:end_date]

# Select relevant columns (O-H-L-C-Adj.C)
data = data[['1. open', '2. high', '3. low', '4. close', '5. volume']]
data.columns = ['open', 'high', 'low', 'close', 'volume']

# Calculate daily returns using the adjusted close prices
#data['daily_return'] = data['adjusted_close'].pct_change()
data.dropna(inplace=True)

# Save the data with daily returns to CSV
data.to_csv('SPY_Daily_Data.csv')

In [8]:
data

,open,high,low,close,volume
date,,,,,
2018-02-01,281.07,283.06,280.680,281.58,90102470.0
2018-02-02,280.08,280.23,275.410,275.45,173174790.0
2018-02-05,273.45,275.85,263.310,263.93,294681816.0
2018-02-06,259.94,269.70,258.700,269.13,355026782.0
2018-02-07,268.50,272.36,267.580,267.67,167376100.0
...,...,...,...,...,...
2025-01-27,594.81,599.69,594.640,599.37,70361125.0
2025-01-28,600.62,605.37,597.250,604.52,44433322.0
2025-01-29,603.72,604.13,599.220,601.81,37177429.0


## 1.

In [17]:
import yfinance as yf
import pandas as pd

# Define the ticker and date range
symbol = 'SPY'
start_date = '2018-02-01'
end_date = '2025-02-01'

# Download daily data from Yahoo Finance
data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False)

data = data.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Adj Close': 'adjusted_close',
    'Volume': 'volume'
})

# Select only the relevant columns
data = data[['open', 'high', 'low', 'close', 'adjusted_close', 'volume']]

# Calculate daily returns based on adjusted close
data['daily_return'] = data['adjusted_close'].pct_change()

# Drop the first row with NaN
data.dropna(inplace=True)

# Save the data to CSV
data.to_csv('SPY_Daily_Data.csv')

[*********************100%***********************]  1 of 1 completed


In [18]:
data

Price,open,high,low,close,adjusted_close,volume,daily_return
Ticker,SPY,SPY,SPY,SPY,SPY,SPY,
Date,,,,,,,
2018-02-02,280.079987,280.230011,275.410004,275.450012,245.220886,173174800,-0.021770
2018-02-05,273.450012,275.850006,263.309998,263.929993,234.965164,294681800,-0.041822
2018-02-06,259.940002,269.700012,258.700012,269.130005,239.594528,355026800,0.019702
2018-02-07,268.500000,272.359985,267.579987,267.670013,238.294739,167376100,-0.005425
2018-02-08,268.010010,268.170013,257.589996,257.630005,229.356583,246449500,-0.037509
...,...,...,...,...,...,...,...
2025-01-27,594.809998,599.690002,594.640015,599.369995,597.572388,70361100,-0.014145
2025-01-28,600.619995,605.369995,597.250000,604.520020,602.706970,44433300,0.008592


## 2.

In [ ]:
# Define the time range
start_date = '2018-02-01'
end_date = '2025-02-01'

# Fetch the daily 2-Year and 10-Year Treasury yields from FRED
dgs2 = web.DataReader('DGS2', 'fred', start_date, end_date)
dgs10 = web.DataReader('DGS10', 'fred', start_date, end_date)

# Combine the two datasets into a single DataFrame
treasury_data = pd.concat([dgs2, dgs10], axis=1)
treasury_data.columns = ['2yr_yield', '10yr_yield']

# Calculate the daily yield spread (10-Year minus 2-Year)
treasury_data['yield_spread'] = treasury_data['10yr_yield'] - treasury_data['2yr_yield']

# Drop any rows with missing values (optional)
treasury_data.dropna(inplace=True)

# Save the data to a CSV file
treasury_data.to_csv('Treasury_Yields.csv')

In [22]:
treasury_data

,2yr_yield,10yr_yield,yield_spread
DATE,,,
2017-07-03,1.41,2.35,0.94
2017-07-05,1.41,2.33,0.92
2017-07-06,1.40,2.37,0.97
2017-07-07,1.40,2.39,0.99
2017-07-10,1.40,2.38,0.98
...,...,...,...
2024-06-25,4.65,4.23,-0.42
2024-06-26,4.71,4.32,-0.39
2024-06-27,4.70,4.29,-0.41


In [26]:
import yfinance as yf
import pandas as pd

# Define symbols and time range
symbols = ['VXX', 'GLD', 'UUP', 'USO', 'QQQ']
start_date = '2018-02-01'
end_date = '2025-02-01'

# Initialize DataFrame to collect all daily returns
all_data = pd.DataFrame()

# Loop through each symbol
for symbol in symbols:
    print(f'Downloading {symbol}...')
    
    # Download data with adjusted close included
    data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False)

    # Rename adjusted close to custom name
    data = data[['Adj Close']].rename(columns={'Adj Close': f'{symbol}_adj_close'})
    
    # Calculate daily return
    data[f'{symbol}_daily_return'] = data[f'{symbol}_adj_close'].pct_change()
    
    # Keep only the daily return column
    data = data[[f'{symbol}_daily_return']]
    
    # Merge into the master DataFrame
    if all_data.empty:
        all_data = data
    else:
        all_data = pd.merge(all_data, data, left_index=True, right_index=True, how='outer')

# Drop rows where all returns are NaN (e.g., holidays)
all_data.dropna(how='all', inplace=True)

# Save to CSV
all_data.to_csv('VXX_GLD_UUP_USO_QQQ_Daily_Returns.csv')

# Preview
print(all_data.head())
print(all_data.tail())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Price      VXX_daily_return GLD_daily_return UUP_daily_return  \
Ticker                                                          
Date                                                            
2018-02-02         0.128594        -0.013118         0.006053   
2018-02-05         0.335160         0.002532         0.004298   
2018-02-06        -0.016963        -0.010496         0.002139   
2018-02-07         0.033027        -0.004706         0.007686   
2018-02-08         0.238886         0.001523         0.000000   

Price      USO_daily_return QQQ_daily_return  
Ticker                                        
Date                                          
2018-02-02        -0.015106        -0.019945  
2018-02-05        -0.020705        -0.039426  
2018-02-06        -0.005482         0.026499  
2018-02-07        -0.025984        -0.012938  
2018-02-08        -0.022635        -0.042195  
Price      VXX_daily_return GLD_daily_return UUP_daily_return  \
Ticker                                

In [27]:
all_data

Price,VXX_daily_return,GLD_daily_return,UUP_daily_return,USO_daily_return,QQQ_daily_return
Ticker,,,,,
Date,,,,,
2018-02-02,0.128594,-0.013118,0.006053,-0.015106,-0.019945
2018-02-05,0.335160,0.002532,0.004298,-0.020705,-0.039426
2018-02-06,-0.016963,-0.010496,0.002139,-0.005482,0.026499
2018-02-07,0.033027,-0.004706,0.007686,-0.025984,-0.012938
2018-02-08,0.238886,0.001523,0.000000,-0.022635,-0.042195
...,...,...,...,...,...
2025-01-27,0.065884,-0.010405,0.000342,-0.020980,-0.029115
2025-01-28,-0.033041,0.008656,0.004108,0.011934,0.014780


## SP500 P/E Ratio Datos

In [29]:
df = pd.read_csv("/Users/ivan/Downloads/sp-500-pe-ratio-price-to-earnings-chart.csv")

In [30]:
import pandas as pd

def filtrar_por_rango_fechas(df):
    """
    Filtra un DataFrame por la columna 'date' dentro del rango '2018-02-01' a '2025-02-21'.

    Args:
        df (pd.DataFrame): El DataFrame a filtrar.

    Returns:
        pd.DataFrame: El DataFrame filtrado.
    """

    # Asegurar que la columna 'date' esté en formato datetime
    df['date'] = pd.to_datetime(df['date'])

    # Definir las fechas de inicio y fin del rango
    fecha_inicio = pd.to_datetime('2018-02-01')
    fecha_fin = pd.to_datetime('2025-02-01')

    # Filtrar el DataFrame por el rango de fechas
    df_filtrado = df[(df['date'] >= fecha_inicio) & (df['date'] <= fecha_fin)]

    return df_filtrado

df = filtrar_por_rango_fechas(df)
df

,date,value
1082,2018-02-01,23.5086
1083,2018-03-01,22.8766
1084,2018-04-01,21.6203
1085,2018-05-01,22.0874
1086,2018-06-01,22.1944
...,...,...
1162,2024-10-01,27.0888
1163,2024-11-01,28.6411
1164,2024-12-01,27.9253
1165,2025-01-01,28.6798
